# Imports

In [ ]:
from morfeus import BuriedVolume
import numpy as np
import pandas as pd
import statistics
from matplotlib import rcParams
import os, re
import get_sum_conformers_v3 as geetum

## Instructions

* This script pulls from an atom numbers spreadsheet as well as a directory with folders for each ligand that contain all of the log files for each. 

* See example Atom numbers spreadsheet and filesystem for more details

* Input is only required in Part 2 and Part 3. All other cells can be run without user input. 

# Determine What Ligands Will Be Parameterized and Where to Save Data

In [ ]:
# Input either 'All Ligands' or 'Specific Ligands' 
pull_parameters_for = 'All Ligands' 

# If 'Specific Ligands', input a list of ligand IDs
Specific_Ligand_IDs = ['8',  '27']

# Populate variable excel_name with the desired name for the parameters file (e.g. "myparameters.xlsx") 
# and filepath where you wnt  (e.g. "~/Desktop/Parameters/")
excel_name = 'Test.xlsx'
excel_filepath = 'Parameter_spreadsheets/'


# Load Atom Numbers and log filenames and filepaths

This notebook is set up to pull parameters for bisphosphine ligands that have been optimized with PdCl2 and single-point calculations have been run on the optimized structure with and without palladium. 

Then create an excel spreadsheet that contains the names of the log files and the atom numbers both with and without Palladium. These will be read in as Pd_anum_df and NoPd_anum_df.

In [ ]:
# New path for ligand folders
Ligands_path = 'DFT_files_atom_nums/Ligand_Calcs/'

# Filepaths for atom numbers spreadsheets
Pd_path = 'DFT_files_atom_nums/SPE_Pd/' #File path for single-point .log files with PdCl2
NoPd_path = 'DFT_files_atom_nums/NoPd/' #File path for single-point .log files without PdCl2
OPT_path = 'DFT_files_atom_nums/OPT/'   #File path for single-point .log files of geometry optimizations


# load in atom numbers spreadsheet
num_columns_Pd = 17   #input the number of columns in the Pd atom number spreadsheet (should not change)
num_columns_NoPd = 15    #input the number of columns in the NoPd atom number spreadsheet (should not change)
Pd_anum_df = pd.read_excel('DFT_files_atom_nums/Atom_numbers_4_2022.xlsx', sheet_name="Pd", usecols=list(range(0,(num_columns_Pd))))
NoPd_anum_df = pd.read_excel('DFT_files_atom_nums/Atom_numbers_4_2022.xlsx', sheet_name="No_Pd", usecols=list(range(0,(num_columns_NoPd))))
# dictionary: key = Ligand ID, value = Ligand Name
ligname_dict = dict(zip(list(Pd_anum_df['Ligand ID']), list(Pd_anum_df.Ligand)))
Pd_anum_df.head()

# Generate dictionary with all log names for all conformers



NO USER INPUT REQUIRED BEYOND THIS CELL

In [ ]:
# Master_Conf_Dict
# key = ligand id (str)
# value = sub dictionary
# sub dictionary:
# key = conformer number (int)
# value = [opt_name, SPE_name, SPE_NoPd_name]
Master_Conf_Dict = {}

# Iterate through ligand folders
ligand_folders = os.listdir(Ligands_path)

# removes items in filenames in the Ligands_path directory that are not folders (e.g. hidden files like .DS_Store)
for item in ligand_folders:
    try:
        os.listdir(Ligands_path + item)
    except:
        ligand_folders.remove(item)

# Takes each folder name and populates with the ligand ID (string element of folder name before '_') and 
# The names of all of the log folders that should be in there. This is done by recognizing the base name (i.e. Segphos)
# and the number of coformers, and generating 'Segphos_#', 'Segphos_#_SPE', and 'Segphos_#_SPE_NoPd' for each conformer
# It is critical that each folder is named '<Ligand ID>_<Ligand Name>' and 
# each log file is named <ligand name>_<conformer number>_<.log, _SPE.log, _SPE_NoPd.log for OPT and single points repsectively>.
for folder in ligand_folders:
    ID_i = int(folder.split('_')[0]) # Ligand ID
    sub_dict = {} 
    conformers = [] # will populate with conformer names (e.g. Segphos_1, Segphos_2, etc.)
    
    # find unique conformer names
    for file in os.listdir(Ligands_path + folder):
        # appends only the optimization files to the conformers list (e.g. Segphos_1, Segphos_2, etc.)
        if re.search('_[12345].log', file) and file[:-4] not in conformers:
            conformers.append(file[:-4])

    # add conformer number (e.g. 1) and conformer log names 
    # (e.g. 'Segphos_1.log', 'Segphos_1_SPE.log', and 'Segphos_1_SPE_NoPd.log')
    # note that the base name is defined by the optimization log file of conformer 1.
    # This means if conformer 1 is named 'Segphos_1.log', the base name will be Segphos.
    # However if the base name has something in it like 'tight', that will be included in the base name for every 
    # Conformer
    for conf_num in range(1,len(conformers)+1): # not zero-indexed
        conf_name = conformers[0][:-2] + '_' + str(conf_num)# name of conformer (e.g. 'Segphos_1')
        conf_opt, conf_SPE, conf_NoPd = conf_name, conf_name + '_SPE', conf_name + '_SPE_NoPd'
        sub_dict[conf_num] = [conf_opt, conf_SPE, conf_NoPd]
    Master_Conf_Dict[ID_i] = sub_dict

# Sort Master_Conf_Dict in order of ascending ligand IDs
Master_Conf_Dict = dict(sorted(Master_Conf_Dict.items()))

## Clean-Up Dictionary with All log Names

* Removes ligands from Master_Conf_Dict if any of their respective log names are not real files

(e.g. if 'Segphos_3.log' is not a real file and the optimization for comformer 3 is actually named 'Segphos_3_tight.log' then an alert will be raised and all coformers of Segphos will be removed from the dictionary)

In [ ]:
Problem_Ligands_Dict = {} # Problem ligands: key = ligand_ID, value = list of non-existant ligand names
        
# iterate through all of the ligand folders
for lig_folder in ligand_folders:
    problem_files = []
    
    # record the ligand ID
    lig_ID = int(lig_folder.split('_')[0] ) 
    
    # make a list of all the files in the given folder
    actual_files = os.listdir(Ligands_path + lig_folder)
    
    # make a list of dictionary files (OPTs, SPEs, and NoPd_SPEs)
    dict_files = [file + '.log' for sublist in Master_Conf_Dict[lig_ID].values() for file in sublist]
    

    # check if every file in the dictionary list is in the actual file list
    for file in dict_files:
        if file not in actual_files:
            problem_files.append(file)
    
    # updates Problem_Ligands_Dict with ligands that have naming issues
    if len(problem_files) > 0:
        Problem_Ligands_Dict[lig_ID] = problem_files 
        
# print out the ligands that have issues
for ligand, files in Problem_Ligands_Dict.items():
    print('--------------------------------')
    print('Problem with ligand {}:'.format(ligand))
    for file in files:
        print(file)
    print('\n\n')
    
# Remove ligands with naming issues from Master_Conf_Dict
for i in Problem_Ligands_Dict:
    del Master_Conf_Dict[i]      

## Check that each folder has the correct ligand ID

* This cell takes the ligand ID from Master_Conf_Dict and makes sure that the SPE, NoPd, and OPT lognames for the lowest energy conformer and recorded in the atom numbers spreadsheet actually exists in that folder

In [ ]:
folder_ids = [folder.split('_')[0] for folder in ligand_folders]
folder_dict = dict(zip(folder_ids, ligand_folders))

folders_with_issues = []
for ID, conf_dict_i in Master_Conf_Dict.items():
    ID = str(ID)
    folder_name_i = folder_dict[ID]
    correct_SPE = list(Pd_anum_df[Pd_anum_df['Ligand ID'] == int(ID)]['log_name'])[0] + '.log'
    correct_NoPd_SPE = list(NoPd_anum_df[NoPd_anum_df['Ligand ID'] == int(ID)]['log_name'])[0] + '.log'
    correct_OPT = list(NoPd_anum_df[NoPd_anum_df['Ligand ID'] == int(ID)]['opt_log_name'])[0] + '.log'
    correct_LOGs = [correct_NoPd_SPE, correct_OPT, correct_SPE]
    
    try:
        real_files = os.listdir(Ligands_path + folder_name_i)
        for correct_filename in correct_LOGs:
            if correct_filename not in real_files:
                if folder_name_i not in folders_with_issues:
                    folders_with_issues.append(folder_name_i)
                print('{} {}.\n{} does not exist in the folder.\n'.format(ID, folder_name_i, correct_filename))
    except:
        print('Problem with {}. This folder does not exist\n\n\n'.format(folder_name))
        

print('{} folders have naming inconsistencies'.format(len(folders_with_issues)))
print(folders_with_issues)


## Creates an atom number dataframe with a row for every conformer of every ligand

In [ ]:
# Ligand_Filepath_Dict key = ID (int) Value = Filepath pointing to ligand log and com files.
Ligand_Filepath_Dict = {}
for folder in ligand_folders:
    ID = int(folder.split('_')[0])
    filepath = Ligands_path + folder + '/'
    Ligand_Filepath_Dict[ID] = filepath
Ligand_Filepath_Dict = dict(sorted(Ligand_Filepath_Dict.items()))
    

# Create a dataframes Conf_Pd_anum_df and Conf_NoPd_anum_df (w/ and w/o PdCl2) 
# with a row for each ligand conformer. An additional row is added for conformer number (1-5)
Conf_Pd_anum_df = pd.DataFrame(columns=list(Pd_anum_df.columns) + ['Conf_Number'])
Conf_NoPd_anum_df = pd.DataFrame(columns=list(NoPd_anum_df.columns) + ['Conf_Number'])


for ID_i, conformers in Master_Conf_Dict.items():
    for conf in conformers:
        # get the row values for the Pd_anum_df (i.e. Ligand ID, Ligand name, atom numbers)
        Pd_anum_row = Pd_anum_df[Pd_anum_df['Ligand ID'] == int(ID_i)].values[0].tolist()
        NoPd_anum_row = NoPd_anum_df[Pd_anum_df['Ligand ID'] == int(ID_i)].values[0].tolist()
        # omit the log name of the lowest energy conformer that is stored in Pd_anum_df
        Pd_anum_row = Pd_anum_row[:-1]
        # omit the lowest energy conformer NoPd_SPE and OPT log names that are stored in Pd_anum_df
        NoPd_anum_row = NoPd_anum_row[:-2]
        
        # update the row from the atom_numbers_spreadsheet to include the the log names for each conformer 
        # as well as the conformer numbers
        [conf_OPT, conf_SPE, conf_NoPd] = Master_Conf_Dict[ID_i][conf]
        #Add filepath info
        filepath = Ligand_Filepath_Dict[ID_i]
        conf_OPT, conf_SPE, conf_NoPd = filepath + conf_OPT, filepath + conf_SPE, filepath + conf_NoPd
        Pd_anum_row = Pd_anum_row + [conf_SPE, conf]
        NoPd_anum_row = NoPd_anum_row + [conf_NoPd, conf_OPT, conf]

        # add the lists as new rows to both the Conf_Pd_anum_df and Conf_NoPd_anum_df
        Conf_Pd_anum_df.loc[len(Conf_Pd_anum_df.index)] = Pd_anum_row
        Conf_NoPd_anum_df.loc[len(Conf_NoPd_anum_df.index)] = NoPd_anum_row

# Updates Ligand ID and conformer number to strings
Conf_Pd_anum_df = Conf_Pd_anum_df.astype({'Ligand ID': str, 'Conf_Number': str})
Conf_NoPd_anum_df = Conf_NoPd_anum_df.astype({'Ligand ID': str, 'Conf_Number': str})

# Updates the Conf_Pd_anum_df and Conf_NoPd_anum_df so that the ligand ID is updated 
# To include the conformer (i.e. for ligand 8 that has two conformers there is a row '8:1' and '8:2')
Conf_Pd_anum_df['Ligand ID'] = Conf_Pd_anum_df['Ligand ID'] + ':' + Conf_Pd_anum_df['Conf_Number']
Conf_NoPd_anum_df['Ligand ID'] = Conf_NoPd_anum_df['Ligand ID'] + ':' + Conf_NoPd_anum_df['Conf_Number']


display(Conf_Pd_anum_df, Conf_NoPd_anum_df)


# Define Functions For Pulling Parameters

## Define Functions to Pull Burried Volume

* To Do: Double-Check to ensure that the quadrants and octants are assigned correctly

In [ ]:
#This function iterates over a dataframe with the following columns:
#1. Ligand ID, 2. Ligand, 3-13 are atom numbers, and 14 is the name of the log file (without .log) 
def Vbur_one_radius(radius, dataframe, plot_images=False, export_parameters=True):
    bv_dataframe = pd.DataFrame(columns=['Ligand ID', 'Ligand','Vbur%', 'NE', 'NW', 'SE', 'SW'])
    for index, row in dataframe.iterrows():
        enantiomer = False
        if str(row['Ligand ID'])[-2:] == '_1':
            enantiomer = True
        try:
            log_file = row['log_name']
            streams, errors = geetum.get_outstreams(log_file)
            log_coordinates = geetum.get_geom(streams)
            elements = np.array([log_coordinates[i][0] for i in range(len(log_coordinates))])
            coordinates = np.array([np.array(log_coordinates[i][1:]) for i in range(len(log_coordinates))])
            metal_anum = row['Pd']          #Pd atom number (1-indexed)

            bv = BuriedVolume(elements, coordinates, metal_anum,
                      excluded_atoms=[row['Cl1'], row['Cl2']],  #Phosphine atom numbers (1-indexed)
                      z_axis_atoms=[row['P1'],row['P2']],     #Chloride atom numbers (1-indexed)
                      xz_plane_atoms=[row['P2']],      #Phosphine oriented east
                     include_hs=True,
                      radius=radius)           # 
            bv.octant_analysis()  
            bv_quads = bv.quadrants['percent_buried_volume']
            bv_octs = bv.octants['percent_buried_volume']
            if enantiomer == False:
                row_i = {'Ligand ID': row['Ligand ID'],'Ligand': row['Ligand'],'Vbur%': 100*bv.fraction_buried_volume,
                         'NE': bv_quads[1], 'NW': bv_quads[2], 'SW':bv_quads[3], 'SE': bv_quads[4],
                        'NE_octant': bv_octs[0], 'NW_octant': bv_octs[1], 'SW_octant': bv_octs[2], 'SE_octant': bv_octs[3]}
            if enantiomer == True:
                row_i = {'Ligand ID': row['Ligand ID'],'Ligand': row['Ligand'],'Vbur%': 100*bv.fraction_buried_volume,
                         'SE': bv_quads[1], 'SW': bv_quads[2], 'NW':bv_quads[3], 'NE': bv_quads[4],
                        'SE_octant': bv_octs[0], 'SW_octant': bv_octs[1], 'NW_octant': bv_octs[2], 'NE_octant': bv_octs[3]}
            if export_parameters==True:
                bv_dataframe = bv_dataframe.append(row_i, ignore_index=True)
            if plot_images==True:
                print('Steric map for ', row['Ligand'], ":")
                bv.plot_steric_map()
        except:
            print('Unable to acquire Vbur parameters for:', row['Ligand'])
    if export_parameters==True:
        name_addon = "_" + str(radius) + '_Ang'
        bv_dataframe.rename(columns={'Vbur%': 'Vbur%'+ name_addon, 'NE': 'NE' + name_addon, 'NW': 'NW' + name_addon,
                                     'SE': 'SE'+ name_addon, 'SW':'SW' + name_addon, 'NE_octant': 'NE_octant' + name_addon,
                                     'NW_octant': 'NW_octant' + name_addon, 'SW_octant': 'SW_octant' + name_addon,
                                    'SE_octant': 'SE_octant' + name_addon,}, inplace=True)
        return(bv_dataframe)


# This function generates %Vbur (total and quadrants) at defined starting and stopping radius and stepsize
# (e.g. from 2.0Å to 4.0Å at 0.5 Å steps) and returns all of them in a single dataframe.
def get_Vbur(start_r, end_r, step_size, dataframe, plot_images=False, enantiomer=False):
    num_steps = int((end_r-start_r)/step_size + 1)
    radii = [start_r + step_size*i for i in range(num_steps)]
    df_lst = []
    for radius in radii:
        df_lst.append(Vbur_one_radius(radius, dataframe, plot_images))
    dfmaster = pd.merge(df_lst[0], df_lst[1])
    for i in range(2,len(df_lst)):
        dfmaster = pd.merge(dfmaster, df_lst[i])
    return(dfmaster)

def plot_quadrants(radius, dataframe):
    Vbur_one_radius(radius, dataframe, plot_images=True, export_parameters=False)

#test_vbur = get_Vbur(2, 7, 1, Pd_anum_df[:10]) 
#test_vbur.head(5)
#writer = pd.ExcelWriter('Parameter_spreadsheets/' + 'Vbur_2.xlsx', engine='xlsxwriter')
#test_vbur.to_excel(writer, sheet_name= 'Vbur')
#writer.save()

## Define Electronics and Bond-Angle Functions

In [ ]:
def bond_vals(dataframe_Pd, dataframe_NoPd):
    bondvals_dataframe_Pd = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry'])#['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry', 'PR1_NBO', 'PR2_NBO', 'PRBack_NBO', 'XR1_NBO', 'XR2_NBO', 'XRBack_NBO', 'P1_NMR', 'X_NMR', 'Homo', 'Lumo', 'dipole', 'P1NBO', 'X_NBO', 'Pd_NBO', 'Cl_1_NBO', 'Cl_2_NBO', 'P1_Pd_bond_occ', 'P1_Pd_bond_eng', 'X_Pd_bond_occ', 'X_Pd_bond_eng', 'P1_Pd_antibond_occ', 'P1_Pd_antibond_eng', 'X_Pd_antibond_occ', 'X_Pd_antibond_eng', 'Pd_LP_1_occ', 'Pd_LP_1_eng', 'Pd_LP_2_occ', 'Pd_LP_2_eng', 'Pd_LP_3_occ', 'Pd_LP_3_eng', 'Pd_LP_4_occ', 'Pd_LP_4_eng', 'P1_R1_bond_occ', 'P1_R1_bond_eng', 'X_R3_bond_occ', 'X_R3_bond_eng', 'P1_R2_bond_occ', 'P1_R2_bond_eng', 'X_R4_bond_occ', 'X_R4_bond_eng', 'P1_Rback_bond_occ', 'P1_Rback_bond_eng', 'X_Rback_bond_occ', 'X_Rback_bond_eng', 'P1_R1_antibond_occ', 'P1_R1_antibond_eng', 'X_R3_antibond_occ', 'X_R3_antibond_eng', 'P1_R2_antibond_occ', 'P1_R2_antibond_eng', 'X_R4_antibond_occ', 'X_R4_antibond_eng', 'P1_Rback_antibond_occ', 'P1_Rback_antibond_eng', 'X_Rback_antibond_occ', 'X_Rback_antibond_eng'])
    bondvals_dataframe_NoPd = pd.DataFrame(columns=['Ligand ID'])#['Ligand ID', 'NoPd_PR1_NBO', 'NoPd_PR2_NBO', 'NoPd_PRBack_NBO', 'NoPd_XR1_NBO', 'NoPd_XR2_NBO', 'NoPd_XRBack_NBO','NoPd_Homo', 'NoPd_Lumo', 'NoPd_dipole', 'NoPd_P1NBO', 'NoPd_X_NBO', 'NoPd_P1_NMR', 'NoPd_X_NMR', 'NoPd_aniso_P1_NMR', 'NoPd_aniso_X_NMR', 'NoPd_P1_LP_occ', 'NoPd_P1_LP_eng', 'NoPd_X_LP_occ', 'NoPd_X_LP_eng', 'NoPd_P1_R1_bond_occ', 'NoPd_P1_R1_bond_eng', 'NoPd_P1_R2_bond_occ', 'NoPd_P1_R2_bond_eng', 'NoPd_P1_Rback_bond_occ', 'NoPd_P1_Rback_bond_eng', 'NoPd_X_R3_bond_occ', 'NoPd_X_R3_bond_eng', 'NoPd_X_R4_bond_occ', 'NoPd_X_R4_bond_eng', 'NoPd_X_Rback_bond_occ', 'NoPd_X_Rback_bond_eng', 'NoPd_P1_R1_antibond_occ', 'NoPd_P1_R1_antibond_eng', 'NoPd_P1_R2_antibond_occ', 'NoPd_P1_R2_antibond_eng', 'NoPd_P1_Rback_antibond_occ', 'NoPd_P1_Rback_antibond_eng', 'NoPd_X_R3_antibond_occ', 'NoPd_X_R3_antibond_eng', 'NoPd_X_R4_antibond_occ', 'NoPd_X_R4_antibond_eng', 'NoPd_X_Rback_antibond_occ', 'NoPd_X_Rback_antibond_eng', 'NoPd_LP_P1_s', 'NoPd_LP_X_s'])
    
    for index, row in dataframe_Pd.iterrows():
        try:
            str_row = row.astype(str)
            atom_num_dict, SPE_dict, measurements_dict = geetum.get_SPE_Pd(str_row['log_name'], [str_row['P1'], str_row['P2']], [str_row['Pd'], str_row['Cl1'], str_row['Cl2']], [str_row['R1'], str_row['R2'], str_row['R3'], str_row['R4']], [str_row['RBack1'], str_row['RBack2']])            
            row_head = {'Ligand ID': row['Ligand ID'], 'Ligand': row['Ligand'], 'Symmetry': row['Symmetry']}
            row_i = {**row_head,**SPE_dict, **measurements_dict}
            bondvals_dataframe_Pd = bondvals_dataframe_Pd.append(row_i, ignore_index=True)
        except:
            print('Unable to acquire bond/angle parameters (w/ PdCl2) for:', row['Ligand'])    
    for index, row in dataframe_NoPd.iterrows():
        try:
            str_row = row.astype(str)
            SPE_NoPd_dict = geetum.get_SPE_NoPd(str_row['log_name'], [str_row['P1'], str_row['P2']], [str_row['R1'], str_row['R2'], str_row['R3'], str_row['R4']], [str_row['RBack1'], str_row['RBack2']])
            polarizability_dict = geetum.get_polarizability(str_row['opt_log_name'])
            row_head = {'Ligand ID': row['Ligand ID']}
            row_i = {**row_head,**SPE_NoPd_dict,**polarizability_dict}
            bondvals_dataframe_NoPd = bondvals_dataframe_NoPd.append(row_i, ignore_index=True)
        except:
            print('Unable to acquire bond/angle parameters (w/o PdCl2) for:', row['Ligand'])
    
    bondvals_dataframe_Pd['Ligand ID'] = bondvals_dataframe_Pd['Ligand ID'].astype(str)
    bondvals_dataframe_NoPd['Ligand ID'] = bondvals_dataframe_NoPd['Ligand ID'].astype(str)
    bondvals_dataframe = pd.merge(bondvals_dataframe_Pd, bondvals_dataframe_NoPd, on='Ligand ID')
    return(bondvals_dataframe)

##Below code is for testing this cell 

#test_bondvals = bond_vals(Conf_Pd_anum_df2, Conf_NoPd_anum_df2)
#test_bondvals.head(20)


# Renumber Atoms Based On X/P–Cback Occupancy

* Input only required above the hashed line

* Specify whether you want to pull parameters for the full library of just for a list of specific ligands

TO DO: 

* Check that this is working correctly by spot checking the parameter values vs the Phosphorus numbering

* Add a cell to ensure that the atom numbers are the same for every conformer of a given compound 

* Check by hand that each ligand of a given backbone has the same P1/P2 designation

* Remove print statements from get_sum.py script

* Excel Hack: copy a column of ligand IDs in excel, paste them into a print(""" """.split()) statement and copy the resultant printed list of strings

In [ ]:
if pull_parameters_for == 'Specific Ligands':
    Conf_Pd_anum_df2 = Conf_Pd_anum_df.loc[Conf_Pd_anum_df.apply(lambda row: row['Ligand ID'].split(':')[0] in Specific_Ligand_IDs, axis=1)]
    Conf_NoPd_anum_df2 = Conf_NoPd_anum_df.loc[Conf_NoPd_anum_df.apply(lambda row: row['Ligand ID'].split(':')[0] in Specific_Ligand_IDs, axis=1)]

if pull_parameters_for == 'All Ligands':
    Conf_Pd_anum_df2, Conf_NoPd_anum_df2 = Conf_Pd_anum_df, Conf_NoPd_anum_df
##########################################################################################################

Pd_anum_df2 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'Pd', 'Cl1', 'Cl2', 'R1', 'R2',
       'R3', 'R4', 'RBack1', 'RBack2', 'Symmetry', 'Chiral', 'log_name'])
NoPd_anum_df2 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'R1', 'R2', 'R3', 'R4',
       'RBack1', 'RBack2', 'Symmetry', 'Chiral', 'log_name', 'opt_log_name'])
bond_vals_df0 = bond_vals(Conf_Pd_anum_df2, Conf_NoPd_anum_df2)
bond_vals_df0 = bond_vals_df0.set_index('Ligand ID')
working_ligands = list(bond_vals_df0.index)   # This returns a list of ligand IDs for ligands that work in bond_vals function  

for index, row in Conf_Pd_anum_df2.iterrows():     #Pd block
    lig_ID = str(row['Ligand ID'])
    P1_Rback_occ = bond_vals_df0.loc[lig_ID,'P1_RBack1_bond_occ']
    P2_Rback_occ = bond_vals_df0.loc[lig_ID,'P2_RBack2_bond_occ']
    if lig_ID not in working_ligands:
        print(lig_ID, ' was not added to new atom number list dued to failed parameterization!')
    else:
        if row['Renumber'] == False:
            #Pd_anum block 44
            row_Pd = row.drop('Renumber')
            Pd_anum_df2.loc[len(Pd_anum_df2.index)] = list(row_Pd[:-1])
        else:
            if P2_Rback_occ > P1_Rback_occ:
                #Pd_anum block 
                Ligand_ID = row['Ligand ID']
                Ligand = row['Ligand']
                P1 = row['P2']
                P2 = row['P1']
                Pd = row['Pd']
                Cl1 = row['Cl2']
                Cl2 = row['Cl1']
                R1 = row['R4']
                R2 = row['R3']
                R3 = row['R2']
                R4 = row['R1']
                RBack1 = row['RBack2']
                RBack2 = row['RBack1']
                Symmetry = row['Symmetry']
                Chiral = row['Chiral']
                log_name = row['log_name']
                new_row = [Ligand_ID, Ligand,  P1, P2, Pd, Cl1, Cl2, R1, R2, R3, R4, RBack1, RBack2, Symmetry, Chiral, log_name]
                Pd_anum_df2.loc[len(Pd_anum_df2.index)] = new_row
            else:
                #Pd_anum block  
                row_Pd = row.drop('Renumber')
                Pd_anum_df2.loc[len(Pd_anum_df2.index)] = list(row_Pd[:-1])                
                
                
for index, row in Conf_NoPd_anum_df2.iterrows():     #NoPd block
    lig_ID = str(row['Ligand ID'])
    P1_Rback_occ = bond_vals_df0.loc[lig_ID,'P1_RBack1_bond_occ']
    P2_Rback_occ = bond_vals_df0.loc[lig_ID,'P2_RBack2_bond_occ']
    if lig_ID not in working_ligands:
        print(lig_ID, ' was not added to new atom number list dued to failed parameterization!')
    else:
        if row['Renumber'] == False:
            row_NoPd = row.drop('Renumber')
            NoPd_anum_df2.loc[len(NoPd_anum_df2.index)] = list(row_NoPd[:-1])
        else:
            if P2_Rback_occ > P1_Rback_occ:
                #NoPd_anum block 
                Ligand_ID = row['Ligand ID']
                Ligand = row['Ligand']
                P1 = row['P2']
                P2 = row['P1']
                R1 = row['R4']
                R2 = row['R3']
                R3 = row['R2']
                R4 = row['R1']
                RBack1 = row['RBack2']
                RBack2 = row['RBack1']
                Symmetry = row['Symmetry']
                Chiral = row['Chiral']
                log_name = row['log_name']
                opt_log_name = row['opt_log_name']
                new_row = [Ligand_ID, Ligand, P1, P2, R1, R2, R3, R4, RBack1, RBack2, Symmetry,  Chiral, log_name, opt_log_name]
                NoPd_anum_df2.loc[len(NoPd_anum_df2.index)] = new_row
            else:
                #NoPd_anum block 
                row_NoPd = row.drop('Renumber')
                NoPd_anum_df2.loc[len(NoPd_anum_df2.index)] = list(row_NoPd[:-1])
    

                
# display(Pd_anum_df2, NoPd_anum_df2) # display not defined

## Check that every conformer for each ligand has the same P1/P2 designation

In [ ]:
# Generate a list of all the ligand IDs for which parameters were pulled.
IDs_Pulled = []
for i in Pd_anum_df2['Ligand ID'].to_list():
    ID = i.split(':')[0]
    if ID not in IDs_Pulled:
        IDs_Pulled.append(ID)
        
# Check that the atom numbers are the same for every row of a given ID
inconsistent_P1_P2_ligands = [] 
for ID in IDs_Pulled:
    # Select all conformers for a given ID
    Sub_df_Pd = Pd_anum_df2.loc[Pd_anum_df2.apply(lambda row: ID == row['Ligand ID'].split(':')[0], axis = 1)]
    Sub_df_NoPd = NoPd_anum_df2.loc[NoPd_anum_df2.apply(lambda row: ID == row['Ligand ID'].split(':')[0], axis = 1)]
    
    # make a list of atom number columns for Pd and NoPd dataframes
    Pd_atom_number_columns, NoPd_atom_number_columns = Pd_anum_df2.columns[2:-3], NoPd_anum_df2.columns[2:-4]
    
    # Check that the atom numbers in each column are the same
    unique_anumbers_Pd = [len(pd.unique(Sub_df_Pd[atom])) for atom in Pd_atom_number_columns]
    unique_anumbers_NoPd = [len(pd.unique(Sub_df_NoPd[atom])) for atom in NoPd_atom_number_columns]
    
    inconsistent_Pd = True in (unique_anumb > 1 for unique_anumb in unique_anumbers_Pd)
    inconsistent_NoPd = True in (unique_anumb > 1 for unique_anumb in unique_anumbers_NoPd)
    
    # Print warning that atom numbers are inconsistent across conformers for a given ligand
    if inconsistent_Pd or inconsistent_NoPd:
        inconsistent_P1_P2_ligands.append(ID)

print('P1/P2 convention is inconsistent for the following ligands:\n')
for ID in inconsistent_P1_P2_ligands:
    print('{} (ID {})'.format(ligname_dict[int(ID)], ID))
      
    

# Generate Atom Number Spreadsheets w/ Enantiomers

In [ ]:
# Renumbering is not required for Vbur parameters. Instead we simply switch the lables on North and South octants
# and quadrants (e.g. NW_4Å becomes SW_4Å)

# Pd_anum_df3 and NoPd_anum_df3 just have extra rows with the same atom numbers
# Pd_anum_df4 and NoPd_anum_df4 have updated atom numbers where R2/3 and R3/4 are switched.

Pd_anum_df3 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'Pd', 'Cl1', 'Cl2', 'R1', 'R2',
       'R3', 'R4', 'RBack1', 'RBack2', 'Symmetry', 'log_name'])
NoPd_anum_df3 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'R1', 'R2', 'R3', 'R4',
       'RBack1', 'RBack2', 'Symmetry', 'log_name',  'opt_log_name'])
Pd_anum_df4 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'Pd', 'Cl1', 'Cl2', 'R1', 'R2',
       'R3', 'R4', 'RBack1', 'RBack2', 'Symmetry', 'log_name'])
NoPd_anum_df4 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'R1', 'R2', 'R3', 'R4',
       'RBack1', 'RBack2','Symmetry', 'log_name', 'opt_log_name'])

##### Pd_anum_df3 and NoPd_anum_df3 #####
# iterates through Pd_anum_df2 and adds rows for both base ligand and enantiomer (e.g. R_Segphos and R_Segphos_ENANT)
# Pd block 
for index, row in Pd_anum_df2.iterrows():   
    if row['Chiral'] == True:
        new_row = list(row)
        del new_row[-2] # Removes Chiral = T/F column from the row.
        Pd_anum_df3.loc[len(Pd_anum_df3.index)] = new_row #Adds row for calculated enantiomer
        new_row[0] = str(new_row[0]) + '_1'
        new_row[1] = new_row[1] + '_ENANT'
        Pd_anum_df3.loc[len(Pd_anum_df3.index)] = new_row #Adds row for new enantiomer (with same atom numbers for now)
    else:
        new_row = list(row)
        del new_row[-2]
        Pd_anum_df3.loc[len(Pd_anum_df3.index)] = new_row
        
# NoPd block         
for index, row in NoPd_anum_df2.iterrows():   
    if row['Chiral'] == True:
        new_row = list(row)
        del new_row[-3]
        NoPd_anum_df3.loc[len(NoPd_anum_df3.index)] = new_row #Adds row for calculated enantiomer
        new_row[0] = str(new_row[0]) + '_1'
        new_row[1] = new_row[1] + '_ENANT'
        NoPd_anum_df3.loc[len(NoPd_anum_df3.index)] = new_row #Adds row for new enantiomer (with same atom numbers for now)
    else:
        new_row = list(row)
        del new_row[-3]
        NoPd_anum_df3.loc[len(NoPd_anum_df3.index)] = new_row

##### Pd_anum_df4 and NoPd_anum_df4 #####
for index, row in Pd_anum_df3.iterrows():  #Pd Block
    if str(row['Ligand ID'])[-2:] == '_1':
        Ligand_ID = row['Ligand ID']
        Ligand = row['Ligand']
        P1 = row['P1']
        P2 = row['P2']
        Pd = row['Pd']
        Cl1 = row['Cl1']
        Cl2 = row['Cl2']
        R1 = row['R2']
        R2 = row['R1']
        R3 = row['R4']
        R4 = row['R3']
        RBack1 = row['RBack1']
        RBack2 = row['RBack2']
        Symmetry = row['Symmetry']
        log_name = row['log_name']
        new_row = [Ligand_ID, Ligand, P1, P2, Pd, Cl1, Cl2, R1, R2, R3, R4, RBack1, RBack2, Symmetry, log_name]
        Pd_anum_df4.loc[len(Pd_anum_df4.index)] = new_row
    else:
        new_row = list(row)
        Pd_anum_df4.loc[len(Pd_anum_df4.index)] = new_row
        
        
for index, row in NoPd_anum_df3.iterrows():  #NoPd Block
    if str(row['Ligand ID'])[-2:] == '_1':
        Ligand_ID = row['Ligand ID']
        Ligand = row['Ligand']
        P1 = row['P1']
        P2 = row['P2']
        R1 = row['R2']
        R2 = row['R1']
        R3 = row['R4']
        R4 = row['R3']
        RBack1 = row['RBack1']
        RBack2 = row['RBack2']
        Symmetry = row['Symmetry']
        log_name = row['log_name']
        opt_log_name = row['opt_log_name']
        new_row = [Ligand_ID, Ligand, P1, P2, R1, R2, R3, R4, RBack1, RBack2, Symmetry, log_name, opt_log_name]
        NoPd_anum_df4.loc[len(NoPd_anum_df4.index)] = new_row
    else:
        new_row = list(row)
        NoPd_anum_df4.loc[len(NoPd_anum_df4.index)] = new_row

# Run Functions to Pull Parameters (Enantiomers)

In [ ]:
Vbur_df = get_Vbur(3, 7, 1.0, Pd_anum_df3[:])      # This must be run on Pd_anum_df3
print('\n\nDone Running Vbur collection')
bond_vals_df = bond_vals(Pd_anum_df4[:], NoPd_anum_df4[:])
print('\n\nDone Running bondvals collection')

## Combine Vbur, and BondVals into a single dataframe

In [ ]:
Vbur_df['Ligand ID'] = Vbur_df['Ligand ID'].astype(str)
df_all_params = pd.merge(bond_vals_df, Vbur_df)

errorcount = 0

# Removes any ligands that have any errors in the parameter acquisition
ligands_with_errors = [] # List of Ligand IDs with Errors
for index, row in df_all_params.iterrows():
    row_lc = [i.lower() for i in row if type(i) == str] # lower case row
    if 'error' in row_lc:
        ID_i = row['Ligand ID'].split(':')[0]
        print('Error detected in {}'.format(row['Ligand ID']))
        if ID_i not in ligands_with_errors:
            ligands_with_errors.append(ID_i)

# Remove ligands where at least one conformer has an error
for index, row in df_all_params.iterrows():
    ID_i = row['Ligand ID'].split(':')[0]
    if ID_i in ligands_with_errors:
        df_all_params.drop(index, inplace=True)            
        
print('The following ligands were removed because they contain one or more errors:\n')
for ID in ligands_with_errors:
    print('{} ({})'.format(ligname_dict[int(ID)], ID))
        
df_all_params[df_all_params.columns[4:]]= df_all_params[df_all_params.columns[4:]].astype(float)

# #generate SPE_G: HF energy from single point
df_all_params['SPE_G'] = df_all_params.apply(lambda row: float(row.SPE_HF_E) + float(row.G) - float(row.OPT_HF_E), axis=1)
df_all_params.head()

Create Conformer dictionaries

* Generates one dictionary for base ligands and one for enantiomers (Enant_Conf_Dict, Base_Conf_Dict)
* Each key is a ligand ID (without enantiomer info) and each value is a list of the specific IDs for all conformers of the ligand within the all_params dataframe. (includes conformer and enantiomer info, e.g. ['8:1_1', '8:2_1', ..]). 

In [ ]:
# All_Base/Enant_Conf: list of all full ids (e.g. '516:1' from the base/enantiomer ligands)
All_Base_Conf = [ID for ID in df_all_params['Ligand ID'] if ID[-2:] != '_1']
All_Enant_Conf = [ID for ID in df_all_params['Ligand ID'] if ID[-2:] == '_1']

# Iterates through both lists and creates Unique_Base_IDs and Unique_Enant_IDs (e.g. '8', instead of '8:1', '8:2', ...)
Unique_Base_IDs = list(set([ID.split(':')[0] for ID in All_Base_Conf]))
Unique_Enant_IDs = list(set([ID.split(':')[0] for ID in All_Enant_Conf])) # do not include '_1' ending

# Create Enant/Base_Conf_Dict:
# Base_Conf_Dict: key = ID (e.g. '8'), value = ID of each conformer (e.g. ['8:1', '8:2', ..])
# Enant_Conf_Dict: key = ID (e.g. '8'), value = ID of each conformer (e.g. ['8:1_1', '8:2_1', ..])
Enant_Conf_Dict, Base_Conf_Dict = {}, {}

# Iterate through Unique_Base_IDs. For ID in Unique_Base_IDs: individual_confs = [] for conf in All_Base_Conf: if ID in conf append id to individual_confs
for ID in Unique_Base_IDs:
    individual_confs = []
    for conf in All_Base_Conf:
        if ID == conf.split(':')[0]:
            individual_confs.append(conf)
    Base_Conf_Dict[ID] = individual_confs
    
for ID in Unique_Enant_IDs:
    individual_confs = []
    for conf in All_Enant_Conf:
        if ID == conf.split(':')[0]:
            individual_confs.append(conf)
    Enant_Conf_Dict[ID] = individual_confs

# Create A Dataframe with Parameters From the Lowest Energy Conformer for Each Ligand (Raw parameters prior to symmetry adaptation)

In [ ]:
# Parameters for the lowest energy conformer for each ligand 
Lowconf_All_Params = pd.DataFrame(columns= [i for i in list(df_all_params.columns) if i not in ['SPE_HF_E', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel']] )


# Lowconf_Params for Base Ligands (i.e. not enantiomers)
for ID, confs in Base_Conf_Dict.items():
    # create a temporary dataframe with all conformers of a given ligand ID
    temp_df = df_all_params[df_all_params['Ligand ID'].isin(confs)]    

    
    # calculate the relative energy within the temporary dataframe
    min_energy = min(list(temp_df.SPE_G))
    temp_df['E_Rel'] = temp_df.apply(lambda row: 627.5*(row.SPE_G - min_energy), axis=1)

    # create the dictionary lowconf_dict
    # This dictionary is created for one ligand and contains the parameters for the lowest energy conformer
    # {'Ligand ID': 8, 'Ligand': 'DIOP', 'Symmetry': 'C2', 'P1_NMR': '254.1', ...}
    lowconf_dict = {'Ligand ID': ID, 'Ligand': ligname_dict[int(ID)], 'Symmetry': temp_df.Symmetry.iloc[0]}
    for param in temp_df.columns:
        if param not in ['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel']:
        
            # This try/except statement is to avoid errors when two conformers have exactly the same relative
            # energy, E = 0. We average the degenerate conformer values.
            try:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param])
            except:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param].mean())
            # this adds the parameter value for param to the lowconf_dict
            lowconf_dict[param] = lowconf_i
            
                        
    # Add low_conf_dict as a row in Lowconf_All_Params
    Lowconf_All_Params.loc[len(Lowconf_All_Params.index)] = lowconf_dict
    
# Lowconf_Params for ENANT Ligands (i.e. not base parameters)
for ID, confs in Enant_Conf_Dict.items():
    # create a temporary dataframe where ligand ID == ID
    temp_df = df_all_params[df_all_params['Ligand ID'].isin(confs)]
    
    # calculate the relative energy for each conformer within the temporary dataframe
    min_energy = min(list(temp_df.SPE_G))
    temp_df['E_Rel'] = temp_df.apply(lambda row: 627.5*(row.SPE_G - min_energy), axis=1)
    
    # lowconf_dict: {'Ligand ID': 8, 'Ligand': 'DIOP', 'Symmetry': 'C2', 'P1_NMR': '254.1', ...}
    lowconf_dict = {'Ligand ID': ID + '_1', 'Ligand': ligname_dict[int(ID)] + '_ENANT', 'Symmetry': temp_df.Symmetry.iloc[0]}
    for param in temp_df.columns:
        if param not in ['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel']:

            # This try/except statement is to avoid errors when two conformers have exactly the same relative
            # energy, E = 0. We average the degenerate conformer values.
            try:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param])
            except:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param].mean())
                
            # Add low_conf_dict as a row in Lowconf_All_Params
            lowconf_dict[param] = lowconf_i
            
                        
    # Add low_conf_dict as a row in Lowconf_All_Params
    Lowconf_All_Params.loc[len(Lowconf_All_Params.index)] = lowconf_dict
    
Lowconf_All_Params.head()
    

# Covert Raw Parameters to Symmetry Adapted Parameters

* sym_adapt_lowconf = Low-Energy Conformer Parameters

In [ ]:
#base parameters are the unique parameters w/o symmetry or min/max/avg
#(e.g. base parameters would be P1_NMR or P2_NMR, not X_NMR_min or X_NMR_max)
base_params = sum(geetum.C1_dict.values(), []) 
ratio_params = list(geetum.ratio_dict.keys())

#Symmetry adapted parameters give min/max/avg for each base parameter
#(e.g. P1_NMR_min, P1_NMR_max, and P1_NMR_avg)
sym_adapt_params = [[i + '_min', i + '_max', i + '_avg'] for i in base_params]
sym_adapt_params = sum(sym_adapt_params, [])

#Symmetry adapted ratios give min/max/avg for each ratiometric parameter
sym_adapt_ratios = [[i + '_min', i + '_max', i + '_avg'] for i in ratio_params]
sym_adapt_ratios = sum(sym_adapt_ratios, [])

# Define funciton to symmetry adapt a dataframe of parameters
def symmetry_adapt(dataframe):

    #sym_adapt_df is a dataframe that will be populated with the symmetry-adapted parameters for each ligand
    sym_adapt_df = pd.DataFrame(columns=['Ligand ID', 'Ligand'] + sym_adapt_params + geetum.global_parameters + sym_adapt_ratios)

    for index, row in dataframe.iterrows():
        if 'Error' in list(row):
            print('1+ error values were found in ligand {} (ID: {})'.format(row['Ligand'], row['Ligand ID']))
            continue
        row_i = {'Ligand ID': row['Ligand ID'], 'Ligand': row['Ligand']}
        for p in geetum.global_parameters: #this keeps the values for parameters not affected by symmetry (HOMO, Pd_LP_E, etc.)
            row_i[p] = row[p]
        if row.Symmetry == 'C2':
            symmetry_dict = geetum.C2_dict
        elif row.Symmetry == 'C2v':
            symmetry_dict = geetum.C2v_dict
        elif row.Symmetry == 'C1':
            symmetry_dict = geetum.C1_dict
        else:
            print('Error, incorrect symmmetry designation for {} (ID: {})'.format(row['Ligand'], row['Ligand ID']))
            continue

        for params in symmetry_dict.values():   #This updates parameters that are affected by symmetry to have min/max/avg vals
            old_vals = [float(row[p]) for p in params] 
            #This gives discrete values for symmetry-equivalent groups 
            for p in params:
                p_min, p_max, p_avg = min(old_vals), max(old_vals), statistics.mean(old_vals)
                row_i[p + '_min'] = p_min
                row_i[p + '_max'] = p_max
                row_i[p + '_avg'] = p_avg
            
        for ratio_i, params in geetum.ratio_dict.items():  #Note: The ratios are optimized for C2 chiral ligands.
            ratio_1 = row[params[0][0]]/row[params[0][1]]
            ratio_2 = row[params[1][0]]/row[params[1][1]]
            ratios = [ratio_1, ratio_2]
            ratio_min, ratio_max, ratio_avg = min(ratios), max(ratios), statistics.mean(ratios)
            row_i[ratio_i + '_min'] = ratio_min
            row_i[ratio_i + '_max'] = ratio_max
            row_i[ratio_i + '_avg'] = ratio_avg
            
        sym_adapt_df = sym_adapt_df.append(row_i, ignore_index=True)

    sym_adapt_df.set_index('Ligand ID', inplace=True)
    return sym_adapt_df

sym_adapt_lowconf = symmetry_adapt(Lowconf_All_Params)


# 10. Create All C2v Dataframe

In [ ]:
def create_all_c2v(dataframe):
#sym_adapt_df is a dataframe that will be populated with the symmetry-adapted parameters for each ligand
    C2v_p_names = [[p + '_min', p + '_max', p + '_avg'] for p in list(geetum.C2v_dict.keys())]
    C2v_p_names = sum(C2v_p_names,[])
    all_c2v_df = pd.DataFrame(columns=['Ligand ID', 'Ligand'] + C2v_p_names + geetum.global_parameters)

    for index, row in dataframe.iterrows():
        if 'Error' in list(row):
            print('1+ error values were found in ligand {} (ID: {})'.format(row['Ligand'], row['Ligand ID']))
            continue
        row_c2v = {'Ligand ID': row['Ligand ID'], 'Ligand': row['Ligand']}
        for p in geetum.global_parameters: #this keeps the values for parameters not affected by symmetry
            row_c2v[p] = row[p]
        C2v_dict = geetum.C2v_dict

        for c2v_param, params in C2v_dict.items():   #This updates parameters that are affected by symmetry to have min/max/avg vals 
            old_vals = [float(row[p]) for p in params] #This gives discrete values for symmetry-equivalent groups 
            
            p_min, p_max, p_avg = min(old_vals), max(old_vals), statistics.mean(old_vals)
            row_c2v[c2v_param + '_min'] = p_min
            row_c2v[c2v_param + '_max'] = p_max
            row_c2v[c2v_param + '_avg'] = p_avg

        all_c2v_df = all_c2v_df.append(row_c2v, ignore_index=True)

    all_c2v_df.set_index('Ligand ID', inplace=True)
    return(all_c2v_df)

all_c2v_lowconf = create_all_c2v(Lowconf_All_Params)
all_c2v_lowconf.head(10)


# Export Parameter Spreadsheets



In [ ]:
writer = pd.ExcelWriter(excel_filepath + excel_name, engine='xlsxwriter')
all_c2v_lowconf.to_excel(writer, sheet_name='all_C2v')
sym_adapt_lowconf.to_excel(writer, sheet_name='Symmetry_Adapted')
writer.save()

